In this notebook, we are going to follow [How convolutional neural networks see the world](https://blog.keras.io/how-convolutional-neural-networks-see-the-world.html) and [Inceptionism: Going Deeper into Neural Networks ](https://ai.googleblog.com/2015/06/inceptionism-going-deeper-into-neural.html) to try to see what happens when we use an existing image recognition system to create an image that has a high probability of being recognized as something it knows.

We will use vgg16 for the network in this notebook and save inception for another try.

In [1]:
# Load the dependencies and the pre-trained vgg16 model.  

import time
import numpy as np
from PIL import Image as pil_image
from keras.preprocessing.image import save_img
from keras import layers
from keras.applications import vgg16
from keras import backend as K

import matplotlib.pyplot as plt
from IPython.display import clear_output, Image, display

# build the VGG16 network with ImageNet weights
#  This time, we load the complete model, not just the conv layers.

vgg = vgg16.VGG16(weights='imagenet', include_top=True)
print('Model loaded.')
vgg.summary()


Using TensorFlow backend.
W0706 07:28:20.858494 140270179895104 deprecation_wrapper.py:119] From /home/joe/tensorflow-1.13_gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0706 07:28:20.865760 140270179895104 deprecation_wrapper.py:119] From /home/joe/tensorflow-1.13_gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0706 07:28:20.866933 140270179895104 deprecation_wrapper.py:119] From /home/joe/tensorflow-1.13_gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0706 07:28:20.880443 140270179895104 deprecation_wrapper.py:119] From /home/joe/tensorflow-1.13_gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is dep

553467904/553467096 [==============================] - 276s 0us/step


W0706 07:32:57.535680 140270179895104 deprecation_wrapper.py:119] From /home/joe/tensorflow-1.13_gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0706 07:32:57.536118 140270179895104 deprecation_wrapper.py:119] From /home/joe/tensorflow-1.13_gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



Model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0      

In [8]:
# we will also use the end labels in order to pick maximum values
import json

with open('../vgg16classes.json', 'r') as f:
    blob = json.load(f)
    
vgg16_dict = {v[1]:k for k,v in blob.items()}

print(list(vgg16_dict.items())[:10])

[('tench', '0'), ('goldfish', '1'), ('great_white_shark', '2'), ('tiger_shark', '3'), ('hammerhead', '4'), ('electric_ray', '5'), ('stingray', '6'), ('cock', '7'), ('hen', '8'), ('ostrich', '9')]


```vgg16_dict``` gives us a dictionary tat maps a text name of an object to the index of the vgg16 class tat represents pictures of that object.  By maximizing the probability that vgg16 sees a particular class, we can invent pictures of these classes of objects (according to the network).